In [1]:
from datasets import load_dataset

In [2]:
ls dataset

caption.txt         Flickr8K_Text/      folder/
Flicker8k_Dataset/  Flickr8k.token.txt  my_test_image/


In [3]:
cat dataset/Flickr8k.token.txt

Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, batulumpur karbonatan, masif, retak-retak, sebagian hancur dan mulai lapuk
Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, masif, retak-retak, sebagian hancur sehingga mulai lapuk dan batulumpur karbonatan
Singkapan batuan sedimen klastik dan batulumpur karbonatan 
batulumpur karbonatan dan Singkapan batuan sedimen klastik
Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas dan batulumpur karbonatan
Pecahan koral
Pecahan koral
Pecahan koral
Pecahan koral
Pecahan koral
Singkapan Batulumpur karbonatan
Singkapan batuan sedimen klastik Batulumpur karbonatan
batulumpur karbonatan dengan bidang perlapisan yang tidak tegas, masif, retak-retak, mulai lapuk dan sebagian hancur 
Singkapan batuan sedimen klastik batulumpur karbonatan dengan bidang perlapisan yang tidak tegas, retak-retak, masif, mulai lapuk dan sebagian hancur
Singkapan batuan sedimen klastik dengan bidan

In [4]:
dataset = load_dataset("text", data_files="dataset/Flickr8k.token.txt", split="train")

Using custom data configuration default-2966e04187faf8ed


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /home/dhiya/.cache/huggingface/datasets/text/default-2966e04187faf8ed/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


In [5]:
dataset["text"]

['Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, batulumpur karbonatan, masif, retak-retak, sebagian hancur dan mulai lapuk',
 'Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas, masif, retak-retak, sebagian hancur sehingga mulai lapuk dan batulumpur karbonatan',
 'Singkapan batuan sedimen klastik dan batulumpur karbonatan ',
 'batulumpur karbonatan dan Singkapan batuan sedimen klastik',
 'Singkapan batuan sedimen klastik dengan bidang perlapisan yang tidak tegas dan batulumpur karbonatan',
 'Pecahan koral',
 'Pecahan koral',
 'Pecahan koral',
 'Pecahan koral',
 'Pecahan koral',
 'Singkapan Batulumpur karbonatan',
 'Singkapan batuan sedimen klastik Batulumpur karbonatan',
 'batulumpur karbonatan dengan bidang perlapisan yang tidak tegas, masif, retak-retak, mulai lapuk dan sebagian hancur ',
 'Singkapan batuan sedimen klastik batulumpur karbonatan dengan bidang perlapisan yang tidak tegas, retak-retak, masif, mulai lapuk dan sebagia

In [6]:
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

In [7]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [8]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [9]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [10]:
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [11]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [12]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [13]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [14]:
encoding = tokenizer.encode("batuan andesit where.")
print(encoding.tokens)

['batuan', 'andesit', 'w', '##h', '##er', '##e', '.']


In [15]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [16]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [17]:
encoding = tokenizer.encode("Andesit berwarna coklat dan abu-abu tua yang bertekstur hornblende dengan tekstur warna abu.")
print(encoding.tokens)

['[CLS]', 'andesit', 'berwarna', 'coklat', 'dan', 'abu', '-', 'abu', 'tua', 'yang', 'bertekstur', 'hornblende', 'dengan', 'tekstur', 'warna', 'abu', '.', '[SEP]']


In [18]:
encoding = tokenizer.encode("andesit merah Let's test this tokenizer...", "batuan on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'andesit', 'merah', 'let', '[UNK]', 's', 't', '##es', '##t', 't', '##h', '##is', '[UNK]', '.', '.', '.', '[SEP]', 'batuan', 'o', '##n', 'a', 'pa', '##ir', 'o', '##f', 'se', '##n', '##t', '##en', '##c', '##es', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [19]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [20]:
tokenizer.decode(encoding.ids)

'andesit merah let s test this... batuan on a pair of sentences.'

In [21]:
tokenizer.save("tokenizer.json")

In [22]:
new_tokenizer = Tokenizer.from_file("tokenizer.json")

In [23]:
new_tokenizer